<a href="https://colab.research.google.com/github/bilmark0/Agile-Manufacturing-TDK-/blob/main/CV/Vision_Transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
import shutil
import random
from google.colab import drive
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow.keras.mixed_precision as mixed_precision
import PIL
import PIL.Image
import sys
from google.colab import files
import math

In [2]:
# Upload the kaggle.json file
uploaded = files.upload()

# Move the uploaded file to the .kaggle directory
kaggle_api_path = os.path.expanduser("~/.kaggle")
if not os.path.exists(kaggle_api_path):
    os.makedirs(kaggle_api_path)

# Ensure the file is set with proper permissions
kaggle_json_path = next(iter(uploaded))  # Get the uploaded filename
os.rename(kaggle_json_path, f"{kaggle_api_path}/kaggle.json")
os.chmod(f"{kaggle_api_path}/kaggle.json", 0o600)

# Download the dataset from Kaggle
!kaggle datasets download -d markbilszky/agile-manufacturing-tdk --unzip

print("Dataset downloaded successfully.")

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/markbilszky/agile-manufacturing-tdk
License(s): unknown
 99% 1.88G/1.89G [00:15<00:00, 160MB/s]
100% 1.89G/1.89G [00:15<00:00, 133MB/s]
Dataset downloaded successfully.


In [3]:
# Set your paths here
base_path = './'  # Replace with the directory containing Reference, error_2, and error_3
new_folder_path = os.path.join(base_path, 'training_data')

# Create the new folder
os.makedirs(new_folder_path, exist_ok=True)

# Move the folders
folders_to_move = ['Reference', 'error_2', 'error_3']
for folder_name in folders_to_move:
    shutil.move(os.path.join(base_path, folder_name), new_folder_path)

print("Folders moved successfully!")

Folders moved successfully!


In [ ]:
# Path to the training data directory
data_dir = './training_data'

# List of main folders to check for nested duplicates
main_folders = os.listdir(data_dir)

for folder in main_folders:
    folder_path = os.path.join(data_dir, folder)

    # Check if this path is indeed a folder
    if os.path.isdir(folder_path):
        # Path to the nested duplicate folder (if exists)
        nested_folder_path = os.path.join(folder_path, folder)

        # Check if a nested folder with the same name exists
        if os.path.isdir(nested_folder_path):
            print(f"Found duplicate folder: {nested_folder_path}")

            # Move all files from nested folder to the main folder
            for filename in os.listdir(nested_folder_path):
                file_path = os.path.join(nested_folder_path, filename)
                target_path = os.path.join(folder_path, filename)

                # Move file to the main folder
                if os.path.isfile(file_path):
                    shutil.move(file_path, target_path)
                    print(f"Moved {file_path} to {target_path}")

            # Remove the nested duplicate folder after moving its contents
            os.rmdir(nested_folder_path)
            print(f"Removed duplicate folder: {nested_folder_path}")

print("Duplicate folders cleaned up successfully.")

In [ ]:
# Sinusoidal learning rate callback
class SinusoidalLearningRate(callbacks.Callback):
    def __init__(self, max_lr, min_lr, total_epochs):
        super(SinusoidalLearningRate, self).__init__()
        self.max_lr = max_lr
        self.min_lr = min_lr
        self.total_epochs = total_epochs

    def on_epoch_begin(self, epoch, logs=None):
        lr = self.min_lr + (self.max_lr - self.min_lr) * (0.5 * (1 + math.sin(2 * math.pi * epoch / self.total_epochs)))
        self.model.optimizer.learning_rate.assign(lr)
        print(f"Epoch {epoch + 1}: Learning rate is {lr:.6f}")

# Hyperparameters for the sinusoidal learning rate
max_lr = 0.002  # Slightly increased max_lr
min_lr = 0.0001
total_epochs = 1000

# Use the sinusoidal learning rate in the callbacks
sinusoidal_lr_callback = SinusoidalLearningRate(max_lr=max_lr, min_lr=min_lr, total_epochs=total_epochs)

data_dir = '/content/training_data'
img_height, img_width = 140, 190

def to_grayscale(image):
    # Convert to grayscale using the formula: Y = 0.2989 R + 0.5870 G + 0.1140 B
    return np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])

# Enhanced data generator with more augmentation
datagen = ImageDataGenerator(
    preprocessing_function=to_grayscale,
    test_split=0.15,
    validation_split=0.15
)

# Training and validation data generators
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=8,
    class_mode='categorical',  # Updated for multi-class classification
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=8,
    class_mode='categorical',
    subset='validation'
)

test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=8,
    class_mode='categorical',
    subset='test'
)

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Define the Vision Transformer model
def create_vit_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(8, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer='l2')(inputs)  # Added L2 regularization
    x = layers.MaxPooling2D((2, 2))(x)

    # Reshape for transformer input
    x = layers.Reshape((-1, x.shape[-1]))(x)

    # Transformer block
    for _ in range(2):  # Number of transformer blocks
        x_res = x
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        attention_output = layers.MultiHeadAttention(num_heads=2, key_dim=8)(x, x)  # Updated key_dim for complexity
        x = layers.Add()([x_res, attention_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        x_res = x
        x = layers.Dense(64, activation='relu', kernel_regularizer='l2')(x)  # Enhanced dense layer
        x = layers.Dropout(0.3)(x)  # Added dropout within transformer block
        x = layers.Dense(x_res.shape[-1])(x)
        x = layers.Add()([x_res, x])

    x = layers.GlobalAveragePooling1D()(x)  # Changed to GlobalAveragePooling1D for consistency
    x = layers.Dense(256, activation='relu', kernel_regularizer='l2')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation='softmax')(x)  # Updated output for 3-class classification

    model = models.Model(inputs, outputs)
    return model

# Create and compile the model
model = create_vit_model((img_height, img_width, 3))
model.compile(optimizer=Adam(learning_rate=max_lr),
              loss='categorical_crossentropy',  # Updated loss for multi-class classification
              metrics=['accuracy'])

# Set up early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1)

# Train the model with the sinusoidal learning rate callback
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=total_epochs,
    callbacks=[early_stopping, model_checkpoint, sinusoidal_lr_callback, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')


Found 17013 images belonging to 3 classes.
Found 1890 images belonging to 3 classes.
Epoch 1: Learning rate is 0.001050
Epoch 1/1000
2127/2127 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.5142 - loss: 1.0378
Epoch 1: val_loss improved from inf to 0.86207, saving model to best_model.keras
2127/2127 ━━━━━━━━━━━━━━━━━━━━ 628s 282ms/step - accuracy: 0.5142 - loss: 1.0377 - val_accuracy: 0.5291 - val_loss: 0.8621 - learning_rate: 0.0010
Epoch 2: Learning rate is 0.001056
Epoch 2/1000
2127/2127 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.5143 - loss: 0.8651
Epoch 2: val_loss improved from 0.86207 to 0.85912, saving model to best_model.keras
2127/2127 ━━━━━━━━━━━━━━━━━━━━ 579s 272ms/step - accuracy: 0.5143 - loss: 0.8651 - val_accuracy: 0.5291 - val_loss: 0.8591 - learning_rate: 0.0011
Epoch 3: Learning rate is 0.001062
Epoch 3/1000
1003/2127 ━━━━━━━━━━━━━━━━━━━━ 4:49 258ms/step - accuracy: 0.5294 - loss: 0.8652